In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score, confusion_matrix
import itertools
import seaborn as sns
import matplotlib.pyplot as plt

def extract_tensor_value(tensor_str):
    """
    Extrai o valor numérico de uma string tensor.
    Ex: 'tensor(0)' -> 0, 'tensor(1)' -> 1
    """
    if isinstance(tensor_str, str) and 'tensor(' in tensor_str:
        return int(tensor_str.split('(')[1].split(')')[0])
    return tensor_str

def load_classifier_results(file_paths, classifier_names):
    """
    Carrega os resultados dos classificadores e organiza em um DataFrame.

    Args:
        file_paths: lista com caminhos dos arquivos CSV
        classifier_names: lista com nomes dos classificadores

    Returns:
        DataFrame com predições de todos os classificadores
    """
    all_predictions = {}

    for i, (file_path, name) in enumerate(zip(file_paths, classifier_names)):
        df = pd.read_csv(file_path)

        # Converte tensor strings para valores numéricos
        df['prediction_clean'] = df['prediction'].apply(extract_tensor_value)
        df['label_clean'] = df['label'].apply(extract_tensor_value)

        if i == 0:
            # Primeira iteração: salva texto e label verdadeiro
            all_predictions['text'] = df['text']
            all_predictions['true_label'] = df['label_clean']

        # Adiciona predições do classificador atual
        all_predictions[f'{name}_pred'] = df['prediction_clean']

    return pd.DataFrame(all_predictions)

def calculate_pairwise_agreement(df, classifier_names):
    """
    Calcula concordância par a par entre todos os classificadores.

    Args:
        df: DataFrame com predições
        classifier_names: lista com nomes dos classificadores

    Returns:
        DataFrame com matriz de concordância (Kappa de Cohen)
    """
    n_classifiers = len(classifier_names)
    kappa_matrix = np.zeros((n_classifiers, n_classifiers))

    # Calcula Kappa para cada par
    for i, j in itertools.combinations(range(n_classifiers), 2):
        name_i = f'{classifier_names[i]}_pred'
        name_j = f'{classifier_names[j]}_pred'

        kappa = cohen_kappa_score(df[name_i], df[name_j])
        kappa_matrix[i, j] = kappa
        kappa_matrix[j, i] = kappa  # Matriz simétrica

    # Diagonal = 1 (concordância perfeita consigo mesmo)
    np.fill_diagonal(kappa_matrix, 1.0)

    return pd.DataFrame(kappa_matrix,
                       index=classifier_names,
                       columns=classifier_names)

def calculate_accuracy_vs_true_labels(df, classifier_names):
    """
    Calcula acurácia de cada classificador vs labels verdadeiros.
    """
    accuracies = {}

    for name in classifier_names:
        pred_col = f'{name}_pred'
        accuracy = (df[pred_col] == df['true_label']).mean()
        accuracies[name] = accuracy

    return accuracies

def plot_agreement_heatmap(kappa_matrix, title="Concordância entre Classificadores (Kappa de Cohen)"):
    """
    Plota heatmap da matriz de concordância.
    """
    plt.figure(figsize=(10, 8))
    sns.heatmap(kappa_matrix,
                annot=True,
                cmap='RdYlBu_r',
                center=0,
                square=True,
                fmt='.3f',
                cbar_kws={"shrink": .8})
    plt.title(title)
    plt.tight_layout()
    plt.show()

def interpret_kappa(kappa):
    """
    Interpreta valores de Kappa de Cohen.
    """
    if kappa < 0:
        return "Concordância pior que o acaso"
    elif kappa < 0.20:
        return "Concordância fraca"
    elif kappa < 0.40:
        return "Concordância razoável"
    elif kappa < 0.60:
        return "Concordância moderada"
    elif kappa < 0.80:
        return "Concordância substancial"
    else:
        return "Concordância quase perfeita"

# Exemplo de uso
def main():
    # CONFIGURAÇÃO - Ajuste estes caminhos e nomes conforme seus arquivos
    file_paths = [
        'bert.csv',
        'roberta.csv',
        'electra_results.csv'
    ]

    classifier_names = ['BERT', 'RoBERTa', 'ELECTRA']

    print("=== ANÁLISE DE CONCORDÂNCIA ENTRE CLASSIFICADORES ===\n")

    # Carrega dados
    print("Carregando dados dos classificadores...")
    df = load_classifier_results(file_paths, classifier_names)
    print(f"Total de amostras: {len(df)}")

    # Calcula concordância par a par
    print("\nCalculando concordância par a par (Kappa de Cohen)...")
    kappa_matrix = calculate_pairwise_agreement(df, classifier_names)

    print("\nMatriz de Concordância (Kappa de Cohen):")
    print(kappa_matrix.round(3))

    # Calcula acurácia vs labels verdadeiros
    print("\nAcurácia vs Labels Verdadeiros:")
    accuracies = calculate_accuracy_vs_true_labels(df, classifier_names)
    for name, acc in accuracies.items():
        print(f"{name}: {acc:.3f}")

    # Análise detalhada par a par
    print("\n=== ANÁLISE DETALHADA PAR A PAR ===")
    for i, j in itertools.combinations(range(len(classifier_names)), 2):
        name_i = classifier_names[i]
        name_j = classifier_names[j]
        kappa = kappa_matrix.loc[name_i, name_j]

        print(f"\n{name_i} vs {name_j}:")
        print(f"  Kappa de Cohen: {kappa:.3f}")
        print(f"  Interpretação: {interpret_kappa(kappa)}")

        # Concordância simples (porcentagem de acordo)
        pred_i = df[f'{name_i}_pred']
        pred_j = df[f'{name_j}_pred']
        simple_agreement = (pred_i == pred_j).mean()
        print(f"  Concordância simples: {simple_agreement:.3f} ({simple_agreement*100:.1f}%)")

    # Plota heatmap
    plot_agreement_heatmap(kappa_matrix)

    # Estatísticas gerais
    print("\n=== ESTATÍSTICAS GERAIS ===")
    # Remove diagonal (concordância consigo mesmo)
    off_diagonal = kappa_matrix.values[~np.eye(len(classifier_names), dtype=bool)]
    print(f"Kappa médio entre classificadores: {off_diagonal.mean():.3f}")
    print(f"Kappa mínimo: {off_diagonal.min():.3f}")
    print(f"Kappa máximo: {off_diagonal.max():.3f}")
    print(f"Desvio padrão: {off_diagonal.std():.3f}")

if __name__ == "__main__":
    main()